# Заместитель (proxy)

Заместитель — это структурный паттерн проектирования, который позволяет подставлять вместо реальных объектов специальные объекты-заменители. Эти объекты перехватывают вызовы к оригинальному объекту, позволяя сделать что-то до или после передачи вызова оригиналу.

## Пример

С ростом популярности сервиса, выросла нагрузка на СУБД. Запросы на SELECT стали обрабатываться слишком долго. Поэтому было принято решение кэшировать результаты запросов на стороне сервиса.

In [15]:
import abc
import uuid
from datetime import datetime
from time import sleep
from typing import Dict


class AbsSQLExecutor:
    @abc.abstractmethod
    def execute(self, query: str) -> str:
        ...


class SQLExecutor(AbsSQLExecutor):
    """
    Этот класс уже был реализован и оттестирован другой командой
    Кэширование требуется внедрить в другом месте
    """

    def execute(self, query: str) -> str:
        """
        Тут должна быть логика SQL-запроса в БД
        """
        return str(uuid.uuid4())


class QueryProxy(AbsSQLExecutor):
    """
    Нужен для добавления кэширования при вызове `execute`
    """

    cached_ttl = 2.0
    exec_timestamp: Dict[str, datetime] = {}
    exec_date: Dict[str, str] = {}

    def __init__(self, sql_executor) -> None:
        self.sql_executor = sql_executor

    @classmethod
    def execute(cls, query: str) -> str:
        now = datetime.now()

        # Если кэш валиден, берем результат из него
        if query in cls.exec_timestamp:
            last_timestamp = cls.exec_timestamp[query]
            elapsed_time = (now - last_timestamp).total_seconds()
            if elapsed_time <= cls.cached_ttl:
                # Тут должно быть получение сохраненных данных в кэше
                return f"EXECUTED AT: {cls.exec_date[query]}"

        cls.exec_timestamp[query] = now
        cls.exec_date[query] = str(uuid.uuid4())
        # Тут должна быть отправка запроса на сервер и сохранение данных в кэше
        return f"EXECUTED AT: {cls.exec_date[query]}"

In [16]:
query = "SELECT ..."

source_selector = SQLExecutor()
proxy_selector = QueryProxy(source_selector)

# Задержки добавлены для демонстрации кэширования
print("Запрос без кэширования")
print(source_selector.execute(query))
print(source_selector.execute(query))

print("А теперь, через кэширующее прокси")
print(proxy_selector.execute(query))
print(proxy_selector.execute(query))
sleep(3)
print("После истечения времени, кэш должен инвалидироваться")
print(proxy_selector.execute(query))

Запрос без кэширования
02aaf790-6917-4570-b816-09729dea8344
81de6c5d-2f5e-4a97-83ab-e665fb885db1
А теперь, через кэширующее прокси
EXECUTED AT: d1b6e6f0-f65d-4892-8ec7-c4d083be1037
EXECUTED AT: d1b6e6f0-f65d-4892-8ec7-c4d083be1037
После истечения времени, кэш должен инвалидироваться
EXECUTED AT: e8fca978-21f9-4f14-89b1-feee38e171d3
